<a href="https://colab.research.google.com/github/Shiv2183/StaffOptimization/blob/main/Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install pyomo
!sudo apt install libglpk-dev python3.6-dev libgmp3-dev
!pip install glpk

In [18]:
!git clone https://github.com/Shiv2183/StaffOptimization.git

Cloning into 'StaffOptimization'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [8]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [10]:
from __future__ import division
from pyomo.opt import SolverFactory

In [11]:
# Reading the data from Excel workbook

In [21]:
demand = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =0)
StaffAvailability = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =1)
Cost = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =2)
ServiceRate = pd.read_excel('/content/StaffOptimization/Staffing+Data.xlsx', sheet_name =3)

In [22]:
demand.head(3)

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942


In [23]:
StaffAvailability.head(3)

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75


In [24]:
Cost.head(3)

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180


In [25]:
ServiceRate.head(3)

,MgAppServedPerMonth
0,40


# Data pre-processing

In [12]:
# Create the required Python data structures for indexes and parameters

# Question 1
The company wants to know the optimised staffing recommendations for the business case described. Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).

**Note:** You can write the mathematical formulation directly in your report.

# Question 2
Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step.

**Expected output:**
Create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier.

In [ ]:
# Creating a model instance

In [ ]:
# Define Pyomo sets and Parameters

In [ ]:
# Decision variables

In [ ]:
# Constraints

In [ ]:
# Objective function

In [ ]:
# Invoking the solver

In [ ]:
# Print the value of the objective function

**Checkpoint 1:** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [ ]:
# Creating dataframe for the results

In [ ]:
# Writing the results in to an Excel sheet

# Question 3

**Worst-case and best-case analysis based on the staffs' availability.**
Assuming that the distribution is the same across all the states,

**3.1 Worst case analysis**

*   3.1.1 What is the optimal number of staff members for the worst case? 
*   3.1.2 What is the percentage of outsourcing for the worst case?
*   3.1.3 What is the average cost per application for the worst case?

**3.2 Best case analysis**


*   3.2.1 What is the optimal number of staff members List item 
for the best case?
*   3.2.2 What is the percentage of outsourcing for the best case? 
*   3.2.3 What is the average cost per application for the best case?

**Expected output:**
For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application.

#### 3.1 Worst Case Analysis
**3.1.1 Optimal number of staff members**

In [ ]:
# Creating a model instance

In [ ]:
# Define Pyomo sets and Parameters

In [ ]:
# Decision variables

In [ ]:
# Constraints

In [ ]:
# Objective function

In [ ]:
# Invoking the solver

In [ ]:
# Print the value of the objective function

**Checkpoint 2: The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.**

In [ ]:
# Creating dataframe for the results

**3.1.2 Percentage of outsourced applications**

In [ ]:
# write your code here

**3.1.3 Average cost per application**

In [ ]:
# write your code here

#### 3.2 Best Case Analysis

**3.2.1 Optimal number of staff members**

In [ ]:
# Creating a model instance

In [ ]:
# Define Pyomo sets and Parameters

In [ ]:
# Decision variables

In [ ]:
# Constraints

In [ ]:
# Objective function

In [ ]:
# Invoking the solver

In [ ]:
# Print the value of the objective function

**Checkpoint 3: The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.**

In [ ]:

# Creating dataframe for the results

**3.2.2 Percentage of outsourced applications**

In [ ]:
# write your code here

**3.2.3 Average cost per application**

In [ ]:
# write your code here

# Question 4
**Creating Visualisations**
Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%).  Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis. Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.